# Projekt 2: Sekundärstrukturen von RNAs

## Name: Dang Quynh Tram Nguyen

**Aufgabe 1: RNAs**

In [1]:
ff = open('miRNA.fasta','r')
fasta_file = ff.readlines()
mi_rna = fasta_file[1]

In [ ]:
# Knoten für Suffix Baum
class Knoten:
    def __init__(self, blatt_str, arg):
        self.blatt = blatt_str
        if isinstance(arg, list):
            assert len(arg) > 1, 'Ein Knoten muss mind. 2 Kinder'
            for i in arg: assert isinstance(i, Knoten), 'Index der Knoten Liste ist kein Knoten'
            self.kinder = arg
        else:
            self.pos = arg
            
    def add(self, knoten : Knoten):
        #assert isinstance(knoten, Knoten)
        self.kinder.append(knoten)
    

reverse Komplenment für ein RNA-Sequenz

In [2]:
def Reverse_komplement(rna_sequenz):
    kom_sequenz = str()
    
    for nu in rna_sequenz:
        if nu == "A": kom_sequenz += "U"
        elif nu == "U": kom_sequenz += "A"
        elif nu == "G": kom_sequenz += "C"
        elif nu == "C": kom_sequenz += "G"
        else: #Falls die Sequenz nicht passende Base enthält
            print(nu, "ist kein Nukleotid für RNA")
            break
    
    #die komplementäre Sequenz wird umgekehrt
    rev_kom_seq = kom_sequenz[::-1]

    return rev_kom_seq

In [3]:
rev_kom_sequenz = Reverse_komplement(mi_rna)
print(mi_rna)
print(rev_kom_sequenz)

GGAGCUUAUCAGAAUCUCCAGGGGUACUUUAUAAUUUCAAAAAGUCCCCCAGGUGUGAUUCUGAUUUGCUUC
GAAGCAAAUCAGAAUCACACCUGGGGGACUUUUUGAAAUUAUAAAGUACCCCUGGAGAUUCUGAUAAGCUCC


**Aufgabe 2: maximale Repeats**

Funktion für Suffix Array und Suffix Dictionary von einer RNA

In [4]:
def Suffix(rna, dict_pos_suffix, list_sort_suffix):
    for i in range(len(rna)):
        suffix = rna[i:len(rna)]
        
        list_sort_suffix.append(suffix)
        dict_pos_suffix[suffix] = i
    list_sort_suffix.sort()

In [5]:
#Suffix Array und Suffix Dictionary von mi_RNA
dict_suffix_rna = dict()
list_sort_rna = list()
Suffix(mi_rna,dict_suffix_rna,list_sort_rna)

#Suffix Array und Suffix Dictionary von reversen Komplement
dict_suffix_rev_kom = dict()
list_sort_rev_kom = list()
Suffix(rev_kom_sequenz,dict_suffix_rev_kom,list_sort_rev_kom)


Funktion für die Suche in Suffix Array

In [6]:
def SASearch(pattern,list_sort_suffix_rna):
    #Es gibt keine Bedingung für die Länge des Patterns
    links = 0
    rechts = len(list_sort_suffix_rna)-1
    #print("rechts",rechts)
    while (rechts - links > 1):
        mitte = int((rechts+links)/2)
        #print("mitte",mitte)
        if pattern <= list_sort_suffix_rna[mitte]: rechts = mitte
        else: links = mitte

    #print("pos",rechts)
    return rechts

Funktion zum Vergleich dem Pattern und dem RNA-Abschnitt

In [7]:
def Match(pattern,rna):
    match = 0
    
    if len(pattern) <= len(rna):
        for i in range(len(pattern)):
            if pattern[i] != rna[i]:
                match = i
                break
            else: match = i + 1 #falls ganzer pattern passt 
    else:
        for i in range(len(rna)):
            if pattern[i] != rna[i]: 
                match = i
                break
            else: match = i + 1
                
    return match        

Funktion zur Suche der Repeats mittels der Funktion SASearch

In [8]:
def repeat_of_a_pattern(pattern,list_sort_rna):
    global dict_suffix_rna
    global dict_suffix_rev_kom
    global list_repeats
    
    
    tem_list_sort_rna = list_sort_rna.copy()
    repeat_list = []

    pos_repeat_in_sarray = SASearch(pattern,tem_list_sort_rna)
    length_repeat = Match(pattern,tem_list_sort_rna[pos_repeat_in_sarray])
    while length_repeat > 3: #nur Repeat mit Länge größer als 3 akzeptiert
        #ein gefundenes Repeat und seine Infos werden unter eine Liste gespeichert und in der gesamte Liste addiert
        #repeat_list enthält: 
            #erste Position des Abschnitt des maximalen Repeats
            #erste Position des komplementären Abschnitt (= Länge der RNA -1 - Position im reverseren Komplement - Länge des Abschnittes + 1)
            #die Länge des Abschnitt
        repeat_list = [dict_suffix_rna[tem_list_sort_rna[pos_repeat_in_sarray]],len(dict_suffix_rev_kom) - dict_suffix_rev_kom[pattern] - length_repeat,length_repeat]
        list_repeats.append(repeat_list)
        
        #Finden weitere Vorkommen
        tem_list_sort_rna.remove(tem_list_sort_rna[pos_repeat_in_sarray])
        pos_repeat_in_sarray = SASearch(pattern,tem_list_sort_rna)
        length_repeat = Match(pattern,tem_list_sort_rna[pos_repeat_in_sarray])
    
    

In [9]:
#Eine Liste von allen Repeats in RNA
#Es enthält alle Liste, die jeweils beinhaltet 
    #erste Position des Abschnitt des maximalen Repeats
    #erste Position des komplementären Abschnitt
    #die Länge des Abschnitt
    
list_repeats = []

In [10]:
for i in range(len(list_sort_rev_kom)):
    repeat_of_a_pattern(list_sort_rev_kom[i],list_sort_rna)

print(list_repeats)
print(len(list_repeats))

[[40, 25, 5], [41, 25, 4], [12, 56, 4], [12, 62, 4], [32, 32, 4], [10, 56, 6], [2, 2, 4], [30, 28, 4], [30, 5, 4], [7, 55, 4], [7, 56, 9], [57, 7, 8], [9, 56, 7], [15, 0, 4], [60, 7, 5], [11, 56, 5], [56, 7, 9], [62, 12, 4], [0, 15, 4], [20, 45, 4], [20, 46, 4], [23, 23, 4], [29, 28, 5], [8, 56, 8], [59, 7, 6], [55, 7, 4], [61, 7, 4], [28, 28, 6], [5, 30, 4], [58, 7, 7], [64, 37, 4]]
31


Ändert die Indizes der Position vom Komplementpaar. Erst die kleinere, dann die größere Stelle. Weil die Reihefolge zweier Abschnitten nicht so wichtig ist, kann sie einfach geändert werden, um klar zu sein und später schnell filtert zu werden

In [11]:
for repeat in list_repeats:
    if repeat[1] < repeat[0]:
        a = repeat[0]
        repeat[0] = repeat[1]
        repeat[1] = a
print(list_repeats)
print(len(list_repeats))

[[25, 40, 5], [25, 41, 4], [12, 56, 4], [12, 62, 4], [32, 32, 4], [10, 56, 6], [2, 2, 4], [28, 30, 4], [5, 30, 4], [7, 55, 4], [7, 56, 9], [7, 57, 8], [9, 56, 7], [0, 15, 4], [7, 60, 5], [11, 56, 5], [7, 56, 9], [12, 62, 4], [0, 15, 4], [20, 45, 4], [20, 46, 4], [23, 23, 4], [28, 29, 5], [8, 56, 8], [7, 59, 6], [7, 55, 4], [7, 61, 4], [28, 28, 6], [5, 30, 4], [7, 58, 7], [37, 64, 4]]
31


Entfernt die Verdoppelung

In [12]:
new_list_repeats = []

for repeat in list_repeats:
    if repeat not in new_list_repeats:
        new_list_repeats.append(repeat)
        
print(new_list_repeats)
print(len(new_list_repeats))

[[25, 40, 5], [25, 41, 4], [12, 56, 4], [12, 62, 4], [32, 32, 4], [10, 56, 6], [2, 2, 4], [28, 30, 4], [5, 30, 4], [7, 55, 4], [7, 56, 9], [7, 57, 8], [9, 56, 7], [0, 15, 4], [7, 60, 5], [11, 56, 5], [20, 45, 4], [20, 46, 4], [23, 23, 4], [28, 29, 5], [8, 56, 8], [7, 59, 6], [7, 61, 4], [28, 28, 6], [7, 58, 7], [37, 64, 4]]
26


Filter 1: längste Abschnitte vom selben Start1 und selben Stop2

########### Methode ist falsch irgendwo

In [13]:
new_list_repeats2 = []

#stop = start + length - 1
#je neue Liste in der Liste enthält:
    #erste Postion der ersten Abschnitt des Repeats
    #letzte Position der zweiten Aschnitt
    #Länge des Repeats
for rep in new_list_repeats:
    new_list_repeats2.append(rep)
    new_rep = new_list_repeats2[-1]
    new_rep[1] += new_rep[2] - 1 #start2 -> stop2
print(new_list_repeats2)
print(len(new_list_repeats2))

[[25, 44, 5], [25, 44, 4], [12, 59, 4], [12, 65, 4], [32, 35, 4], [10, 61, 6], [2, 5, 4], [28, 33, 4], [5, 33, 4], [7, 58, 4], [7, 64, 9], [7, 64, 8], [9, 62, 7], [0, 18, 4], [7, 64, 5], [11, 60, 5], [20, 48, 4], [20, 49, 4], [23, 26, 4], [28, 33, 5], [8, 63, 8], [7, 64, 6], [7, 64, 4], [28, 33, 6], [7, 64, 7], [37, 67, 4]]
26


Funktion um das längste Länge von einem Ende zu finden. Diese werden 2 mal genutzt für die Start- und Stopenden der Abschnitten

In [14]:
def max_rep_on_an_end(list_repeats):
    #Sortiert abwärts
    new_list_repeats2_sort = sorted(new_list_repeats2, key=lambda x: (x[0],x[1],x[2]),reverse=True)
    #print(new_list_repeats2_sort)
    #print(len(new_list_repeats2_sort))
    
    #Filter
    new_list_repeats2_filter = [new_list_repeats2_sort[0]]

    for i in range(1,len(new_list_repeats2_sort)):
        pre_rep = new_list_repeats2_sort[i-1]
        rep = new_list_repeats2_sort[i]
        if pre_rep[0] != rep[0] or pre_rep[1] != rep[1]:
            new_list_repeats2_filter.append(rep)
    #print(new_list_repeats2_filter)
    #print(len(new_list_repeats2_filter))
    
    return new_list_repeats2_filter

In [15]:
new_list_repeats2_filter = max_rep_on_an_end(new_list_repeats2)
print(new_list_repeats2_filter)
print(len(new_list_repeats2_filter))

[[37, 67, 4], [32, 35, 4], [28, 33, 6], [25, 44, 5], [23, 26, 4], [20, 49, 4], [20, 48, 4], [12, 65, 4], [12, 59, 4], [11, 60, 5], [10, 61, 6], [9, 62, 7], [8, 63, 8], [7, 64, 9], [7, 58, 4], [5, 33, 4], [2, 5, 4], [0, 18, 4]]
18


In [16]:
for rep in new_list_repeats2_filter:
    print(rep[0],mi_rna[rep[0]:rep[0]+rep[2]],mi_rna[rep[1]-rep[2]+1:rep[1]+1],rep[1])

37 CAAA UUUG 67
32 AAUU AAUU 35
28 UUAUAA UUAUAA 33
25 ACUUU AAAGU 44
23 GUAC GUAC 26
20 GGGG CCCC 49
20 GGGG CCCC 48
12 AAUC GAUU 65
12 AAUC GAUU 59
11 GAAUC GAUUC 60
10 AGAAUC GAUUCU 61
9 CAGAAUC GAUUCUG 62
8 UCAGAAUC GAUUCUGA 63
7 AUCAGAAUC GAUUCUGAU 64
7 AUCA UGAU 58
5 UUAU AUAA 33
2 AGCU AGCU 5
0 GGAG CUCC 18


Filter 2: längsten Abschnitte vom selben Stop1 und selben Start2

########## Methode ist falsch irgendwo

In [17]:
new_list_repeats3 = []

#stop = start + length - 1
#je neue Liste in der Liste enthält:
    #letze Postion der ersten Abschnitt des Repeats
    #erste Position der zweiten Aschnitt
    #Länge des Repeats
for rep in new_list_repeats2_filter:
    new_list_repeats3.append(rep)
    new_rep = new_list_repeats3[-1]
    new_rep[0] += new_rep[2] - 1 #start1 -> stop1
    new_rep[1] = new_rep[1] - new_rep[2] + 1 #stop2 -> start2
    
print(new_list_repeats3)
print(len(new_list_repeats3))

[[40, 64, 4], [35, 32, 4], [33, 28, 6], [29, 40, 5], [26, 23, 4], [23, 46, 4], [23, 45, 4], [15, 62, 4], [15, 56, 4], [15, 56, 5], [15, 56, 6], [15, 56, 7], [15, 56, 8], [15, 56, 9], [10, 55, 4], [8, 30, 4], [5, 2, 4], [3, 15, 4]]
18


In [18]:
new_list_repeats3_filter = max_rep_on_an_end(new_list_repeats3)
print(new_list_repeats3_filter)
print(len(new_list_repeats3_filter))

[[40, 64, 4], [35, 32, 4], [33, 28, 6], [29, 40, 5], [28, 33, 5], [26, 23, 4], [25, 44, 4], [23, 46, 4], [23, 45, 4], [15, 62, 4], [15, 56, 9], [10, 55, 4], [8, 30, 4], [7, 64, 8], [5, 2, 4], [3, 15, 4]]
16


In [19]:
for rep in new_list_repeats3_filter:
    print(rep[0]-rep[2]+1,mi_rna[rep[0]-rep[2]+1:rep[0]+1],mi_rna[rep[1]:rep[1]+rep[2]],rep[1])

37 CAAA UUUG 64
32 AAUU AAUU 32
28 UUAUAA UUAUAA 28
25 ACUUU AAAGU 40
24 UACUU AUUUC 33
23 GUAC GUAC 23
22 GGUA UCCC 44
20 GGGG CCCC 46
20 GGGG CCCC 45
12 AAUC GAUU 62
7 AUCAGAAUC GAUUCUGAU 56
7 AUCA UGAU 55
5 UUAU AUAA 30
0 GGAGCUUA UUUGCUUC 64
2 AGCU AGCU 2
0 GGAG CUCC 15


**Aufgabe 3: RNA Sekundärstrukturen**

Filtern die Überlappungsrepeats und die Repeats, deren Abstand zwischen seinen Abschnitten kleiner als 3 bp ist

In [20]:
#Nicht überlappt Repeats und deren Abschnitt mind. 3 bp voneinander
#Repeat: [stop1,start2,length]

list_max_repeats = new_list_repeats3_filter.copy()
min_gap = 3
list_max_repeats_filter = [rep for rep in list_max_repeats if rep[1] > rep[0]+rep[2]-1 + min_gap]
print(list_max_repeats_filter)
print(len(list_max_repeats_filter))

[[40, 64, 4], [29, 40, 5], [25, 44, 4], [23, 46, 4], [23, 45, 4], [15, 62, 4], [15, 56, 9], [10, 55, 4], [8, 30, 4], [7, 64, 8], [3, 15, 4]]
11


Abstand zwischen zweier Abschnitten von den Repeats

In [21]:
#length = stop - start + 1 (bp)
#gap = start2 - stop1 - 1
    
#Dictionary des Abstands jedes maximalen Repeatpaares in RNA
list_gap_repeats = []

for repeat in list_max_repeats_filter:
    
    gap_repeat = repeat[1] - repeat[0] - 1
    list_gap_repeats.append(gap_repeat)
        
print(list_gap_repeats)
print(len(list_gap_repeats))

[23, 10, 18, 22, 21, 46, 40, 44, 21, 56, 11]
11


In [26]:
for rep in list_max_repeats_filter:
    print(rep[0]-rep[2]+1,mi_rna[rep[0]-rep[2]+1:rep[0]+1],mi_rna[rep[1]:rep[1]+rep[2]],rep[1],"   abstand =",list_gap_repeats[list_max_repeats_filter.index(rep)])

37 CAAA UUUG 64    abstand = 23
25 ACUUU AAAGU 40    abstand = 10
22 GGUA UCCC 44    abstand = 18
20 GGGG CCCC 46    abstand = 22
20 GGGG CCCC 45    abstand = 21
12 AAUC GAUU 62    abstand = 46
7 AUCAGAAUC GAUUCUGAU 56    abstand = 40
7 AUCA UGAU 55    abstand = 44
5 UUAU AUAA 30    abstand = 21
0 GGAGCUUA UUUGCUUC 64    abstand = 56
0 GGAG CUCC 15    abstand = 11


Die Stems bzw. maximalen Repeats laut der RNAfold Webserver

In [ ]:
#Repeat[stop1,start2,length]
stem_struc_web = [[7,56,9],[20,46,4],[25,40,5]]